In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from google.colab import files

In [2]:
uploaded = files.upload()

Saving Housing.csv to Housing (1).csv


In [3]:
df = pd.read_csv('Housing.csv')
print(df.head()) # Переглянути перші 5 рядків таблиці

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [4]:
X = df[['area', 'bathrooms', 'bedrooms']].values  # матриця ознак
y = df['price'].values  # вектор значень

X = np.c_[np.ones(X.shape[0]), X]  # додаємо стовпець одиничних значень

In [5]:
# напишіть функцію гіпотези лінійної регресії у векторному вигляді;
def hypothesis(X, w):
    return np.dot(X, w)

In [6]:
# створіть функцію для обчислення функції втрат у векторному вигляді;
def loss_function(X, y, w):
    m = len(y)
    predictions = hypothesis(X, w)
    return (1 / (2 * m)) * np.sum((predictions - y) ** 2)

In [7]:
# реалізуйте один крок градієнтного спуску;
# Одна ітерація
def gradient_step(X, y, w, alpha):
    m = len(y)
    predictions = hypothesis(X, w)
    gradient = (1 / m) * np.dot(X.T, (predictions - y))
    w = w - alpha * gradient
    return w

In [8]:
# Пошук найкращих параметрів за допомогою градієнтного спуску:
def gradient_descent(X, y, w, alpha, iterations):
    for i in range(iterations):
        w = gradient_step(X, y, w, alpha)
    return w

In [9]:
#  Аналітичне рішення:
def analytical_solution(X, y):
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

In [10]:
# Перевірка результатів за допомогою scikit-learn:
def compare_with_sklearn(X, y):
    model = LinearRegression()
    model.fit(X, y)
    return model.coef_, model.intercept_